In [127]:
# http://gorayni.blogspot.com/2014/12/factor-analysis.html

import os
import numpy as np
from sklearn.decomposition import PCA

#basedir = os.path.abspath(os.path.dirname(__file__)) + "/"
filename = 'gopro_raw_data_weighted_v1.csv'
# weighted3 = weight * raw_data
# weighted2 = (weight * raw_data) * sqrt(weight)
# weighted =  (raw_data(i) - raw_data(mean)) * sqrt(weight)  ***matches SPSS output
X = np.asmatrix(np.genfromtxt(filename, delimiter=',', skip_header=1))
names = list(np.genfromtxt(filename, delimiter=',', names=True).dtype.names)
X_df = pd.DataFrame(X, columns=names)

n_components = 32
pca = PCA(n_components=n_components)
pca.fit(X_df)

factor_matrix = pca.components_.T
factor_matrix_df = pd.DataFrame(factor_matrix)

print factor_matrix.shape
print "sample row: ", factor_matrix[0]

filename_2 = 'gopro_weighted2_PCA_df.csv'
factor_matrix_df.to_csv(filename_2, index=True)

(148, 32)
sample row:  [-0.0168      0.09261053 -0.01422944  0.00085329 -0.15273385  0.03755834
  0.02318159 -0.00445957  0.00330105 -0.08116464  0.08714743 -0.09668426
  0.06979127 -0.09501983  0.03218708 -0.13780884  0.15703218 -0.00677589
 -0.06745426  0.00915324 -0.07446115  0.05752603 -0.03881522  0.11161794
 -0.05273484  0.08420188 -0.04607607  0.07339843 -0.04730323  0.05731678
 -0.1815016   0.04289703]


In [136]:
#following http://sebastianraschka.com/Articles/2014_pca_step_by_step.html
X_cor_mat = np.corrcoef(X.T)

# eigenvectors and eigenvalues for the from the covariance matrix
eig_val_cor, eig_vec_cor = np.linalg.eig(X_cor_mat)

# filter for eigvec > 1.0, add to factor list
eig_vec_cors = []; eig_val_cors = []

print eig_vec_cor[:,1].shape
for i in range(len(eig_val_cor)):
    eigvec_cor = eig_vec_cor[:,i].reshape(1,148).T
    if eig_val_cor[i] >=1:
        #print('Eigenvector {}: \n{}'.format(i+1, eigvec_cov))
        print('Eigenvalue {} from correlation matrix: {}'.format(i+1, eig_val_cor[i]))
        print(40 * '-')
        
        eig_vec_cors.append(eigvec_cor)
        eig_val_cors.append(eig_val_cor[i])

print 'eigenvectors:', len(eig_vec_cors), len(eig_vec_cors[0])
print 'eigenvalues:', len(eig_val_cors), eig_val_cors, 

(148,)
Eigenvalue 1 from correlation matrix: 24.1576881978
----------------------------------------
Eigenvalue 2 from correlation matrix: 7.44693208592
----------------------------------------
Eigenvalue 3 from correlation matrix: 5.46473447211
----------------------------------------
Eigenvalue 4 from correlation matrix: 4.94314906025
----------------------------------------
Eigenvalue 5 from correlation matrix: 3.68635160418
----------------------------------------
Eigenvalue 6 from correlation matrix: 3.434316545
----------------------------------------
Eigenvalue 7 from correlation matrix: 2.78967416025
----------------------------------------
Eigenvalue 8 from correlation matrix: 2.55600869513
----------------------------------------
Eigenvalue 9 from correlation matrix: 2.52687122754
----------------------------------------
Eigenvalue 10 from correlation matrix: 2.35669277993
----------------------------------------
Eigenvalue 11 from correlation matrix: 2.18777843394
-----------

In [242]:
# flatten eigenvector list, convert to matrix
import itertools

eigenvectors = [list(itertools.chain.from_iterable(vector)) for vector in eig_vec_cors]
#print eig_vec_cors[0]
#eigen_vector_matrix = np.asmatrix(eig_vec_cors)
#one_eig_vec_cors = list(itertools.chain.from_iterable(eig_vec_cors[0]))
#print len(one_eig_vec_cors)
print len(eigenvectors)
eigenvector_matrix = np.asmatrix(eigenvectors) 
print eigenvector_matrix.shape
print len(eig_val_cors)
#eig_val_cors2 = np.asarray(eig_val_cors)
#eigenvector_matrix * eig_val_cors2.T

32
(32, 148)
32


ValueError: shapes (32,148) and (1,32) not aligned: 148 (dim 1) != 1 (dim 0)

In [185]:
# data = [1980, 148] eigenvector_matrix = [148, 32] factor_matrix = [148,32]

In [243]:
# now calculate factor loadings (non-roatated)
# http://stats.stackexchange.com/questions/143905/loadings-vs-eigenvectors-in-pca-when-to-use-one-or-another
# print 'eigenvectors:', len(eig_vec_cors), len(eig_vec_cors[0])
# print 'eigenvalues:', len(eig_val_cors), eig_val_cors,

loading_factors_2 = [(np.sqrt(eig_val_cors[i]) * eigenvector_matrix[i,:]).tolist()[0] for i in range(len(eig_val_cors))]
loading_factors_2_matrix = np.asmatrix(loading_factors_2)
print loading_factors_2_matrix.shape

#loading_factors = []
loading_factors_matrix = np.zeros(eigenvector_matrix.shape)
#print loading_factors_matrix_3.shape

for i in range(len(eig_val_cors)):
    z = eigenvector_matrix[i,:] # direction
    y = np.sqrt(eig_val_cors[i]) # scale
    loading_factors_matrix[i,:] = (z * y) # append one column
    # loading_factor = (z * y).tolist()[0]
    # print loading_factor.shape
    # loading_factors.append(loading_factor)

# print len(loading_factors)
# print loading_factors[0].shape
# print type(loading_factors[0])
# print 'checksum:', sum(loading_factors[0])

# loading_factors_matrix = np.asmatrix(loading_factors)
print loading_factors_matrix.shape
print 'checksum', sum(loading_factors_matrix[0,:])

# print loading_factors_matrix_3.shape

# http://stackoverflow.com/questions/14372613/efficient-creation-of-numpy-arrays-from-list-comprehension-and-in-general
#(loading_factors_matrix_4[i,:] = (np.sqrt(eig_val_cors[i]) * eigenvector_matrix[i,:]) for i in range(len(eig_val_cors)))
# loading_factors_matrix_4 = np.fromiter((np.sqrt(eig_val_cors[i]) * eigenvector_matrix[i,:] for i in range(len(eig_val_cors))),int)
#z = eigenvector_matrix[0,:] 
#y = np.sqrt(eig_val_cors[0])
#print "eigenvector:", z.shape # direction
#print "sqrt(eigenvalue):", y # scale
#print "eigenvector * sqrt(eigenvalue):", (z*y).shape, sum((z * y).tolist()[0]) 
# order (y*z) to match excel spss workbook then do pairwise comparisons to validate
# (1, 148)
# 4.9150471206
# sum of eigenvector * sqrt(eigenvalue) = -51.9290001362 # matches excel spss output!

(32, 148)
(32, 148)
checksum -51.9290001362


In [98]:
factor_matrix_df.index = names

In [414]:
# (raw_data(i) - raw_data(mean)) * sqrt(weight)
# http://stats.stackexchange.com/questions/113485/weighted-principal-components-analysis

unweighted_filename = '/Users/pniessen/Rosetta_Desktop/Segmentation_2-point-0/sample_case_work/GoPro/gopro_raw_data_v1.csv'
weights_filename = '/Users/pniessen/Rosetta_Desktop/Segmentation_2-point-0/segmentr/web/gopro_weights.csv'
X_unweighted = np.asmatrix(np.genfromtxt(unweighted_filename, delimiter=',', skip_header=1))
X_weights = np.asmatrix(np.genfromtxt(weights_filename, delimiter=',', skip_header=1))

#print X_unweighted.shape, type(X_unweighted)
#print X_weights.shape, type(X_weights)
#X_unweighted[0,:].mean()
X_unweighted_mean = (np.mean(X_unweighted, axis = 0))
#print X_unweighted_mean.shape
X_unweighted_centered = X_unweighted - X_unweighted_mean
#print X_unweighted_centered.shape
X_weights_sqrt = np.sqrt(X_weights)
#print X_weights_sqrt.shape
#print (X_unweighted_centered[:,0] * X_weights_sqrt)
X_weighted_2 = np.multiply(X_unweighted_centered.T,np.sqrt(X_weights))
print X_weighted_2.shape
#print z[0,0], z[0,1], z[0,2]
#print z[0,0], z[1,0], z[2,0]

weighted_filename = 'gopro_raw_data_weighted_v1.csv'
X_weighted = np.asmatrix(np.genfromtxt(weighted_filename, delimiter=',', skip_header=1))
print X_weighted.shape

print X_weighted_2[0,0], X_weighted_2[0,1], X_weighted_2[0,2]
print X_weighted[0,0], X_weighted[0,1], X_weighted[0,2]
if np.isclose(X_weighted.T, X_weighted_2).any():
    print "X_weighted are similar"

(148, 1980)
(1980, 148)
1.76612021656 -1.81811044543 -2.79458148384
1.76476114 1.375213278 1.47015478
X_weighted are similar


In [418]:
import os
import numpy as np

# input raw data, center, apply weights
# (raw_data(i) - raw_data(mean)) * sqrt(weight)
# http://stats.stackexchange.com/questions/113485/weighted-principal-components-analysis

unweighted_filename = '/Users/pniessen/Rosetta_Desktop/Segmentation_2-point-0/sample_case_work/GoPro/gopro_raw_data_v1.csv'
weights_filename = '/Users/pniessen/Rosetta_Desktop/Segmentation_2-point-0/segmentr/web/gopro_weights.csv'
X_unweighted = np.asmatrix(np.genfromtxt(unweighted_filename, delimiter=',', skip_header=1))
X_weights = np.asmatrix(np.genfromtxt(weights_filename, delimiter=',', skip_header=1))

X_unweighted_mean = (np.mean(X_unweighted, axis = 0))
X_unweighted_centered = X_unweighted - X_unweighted_mean
X_weighted = np.multiply(X_unweighted_centered.T,np.sqrt(X_weights))
X = X_weighted.T

filename = 'gopro_raw_data_weighted_v1.csv'
# weighted3 = weight * raw_data
# weighted2 = (weight * raw_data) * sqrt(weight)
# weighted = (raw_data(i) - raw_data(mean)) * sqrt(weight)  ***matches SPSS output
#X = np.asmatrix(np.genfromtxt(filename, delimiter=',', skip_header=1))
#print "X shape: ", X.shape
X_names = list(np.genfromtxt(filename, delimiter=',', names=True).dtype.names)

#following http://sebastianraschka.com/Articles/2014_pca_step_by_step.html
X_cor_mat = np.corrcoef(X.T)

# eigenvectors and eigenvalues for the from the covariance matrix
eig_val_cor, eig_vec_cor = np.linalg.eig(X_cor_mat)

# filter for eigvec > 1.0, generate loading factor, add to loading_factors_matrix
eig_vec_cors = []; eig_val_cors = []
eig_vec_cors_length = eig_vec_cor[:,1].shape[0]
loading_factors_matrix_2 = np.asmatrix(np.zeros((0,eig_vec_cors_length))) # matrix shape must be tupple

for i in range(len(eig_val_cor)):
    eigvec_cor = eig_vec_cor[:,i].reshape(1,eig_vec_cors_length).T
    if eig_val_cor[i] >=1:
        #print('Eigenvector {}: \n{}'.format(i+1, eigvec_cov))
        #print('Eigenvalue {} from correlation matrix: {}'.format(i+1, eig_val_cor[i]))
        #print(40 * '-')
        
        # can we calculate loading factors right here and add to new matrix with np.append
        eig_vec_cors.append(eigvec_cor) # direction
        eig_val_cors.append(eig_val_cor[i]) # scale sqrt()
        loading_factor = eigvec_cor * np.sqrt(eig_val_cor[i])
        #print "loading_factors_matrix_2 shape: ", loading_factors_matrix_2.shape
        #print "loading_factor shape: ", loading_factor.shape
        loading_factors_matrix_2 = np.r_[loading_factors_matrix_2,loading_factor.T]
        #print "loading_factors_matrix_2 shape after append: ", loading_factors_matrix_2.shape

print 'eigenvectors:', len(eig_vec_cors), len(eig_vec_cors[0])
print 'eigenvalues:', len(eig_val_cors), eig_val_cors, 

# flatten eigenvector list, convert to matrix
import itertools

eigenvectors = [list(itertools.chain.from_iterable(vector)) for vector in eig_vec_cors]
print len(eigenvectors)
eigenvector_matrix = np.asmatrix(eigenvectors)

# now calculate loading factors matrix
loading_factors_matrix = np.zeros(eigenvector_matrix.shape)
#print loading_factors_matrix.shape

for i in range(len(eig_val_cors)):
    z = eigenvector_matrix[i,:] # direction
    y = np.sqrt(eig_val_cors[i]) # scale
    loading_factors_matrix[i,:] = (z * y)

print loading_factors_matrix.shape 
loading_factors_matrix_df = pd.DataFrame(loading_factors_matrix.T)
loading_factors_matrix_df.index = X_names
loading_factors_matrix_df

filename_2 = 'gopro_loading_factors_matrix_df.csv'
loading_factors_matrix_df.to_csv(filename_2, index=True)

for i in range(loading_factors_matrix.shape[0]):
    if not np.isclose(sum(loading_factors_matrix[i,:]),np.sum(loading_factors_matrix_2[i,:])):
        print 'checksum', sum(loading_factors_matrix[i,:])
        print 'checksum_2', np.sum(loading_factors_matrix_2[i,:])
        print 'difference:', sum(loading_factors_matrix[i,:]) - np.sum(loading_factors_matrix_2[i,:])
        print '-----------------'
print loading_factors_matrix.shape
print loading_factors_matrix_2.shape

# test to see if methods are same
if np.isclose(loading_factors_matrix, loading_factors_matrix_2).any():
    print "loading_factors_matrix are similar"
# print 'df_checksum', sum (loading_factors_matrix_df[0])
# loading_factors_matrix_checksum = [sum(loading_factors_matrix[i,:]) for i in range(loading_factors_matrix.shape[0])]
# print loading_factors_matrix_checksum

eigenvectors: 32 148
eigenvalues: 32 [24.155757104811965, 7.447113037334975, 5.4649400779285093, 4.9439310811318427, 3.6865391339364235, 3.4340997519313503, 2.7900305621016268, 2.5559640103493169, 2.5267444321276407, 2.3568300983347998, 2.1878297057233067, 2.0391067007640133, 1.8519169984271135, 1.7450205521411941, 1.6653690568723831, 1.555346555980647, 1.5388543760013815, 1.4881027933336286, 1.4457310452464551, 1.415433092928408, 1.3539280197718768, 1.2928889438417752, 1.2335816262655197, 1.2596567344409846, 1.2050707768655422, 1.1709719274729844, 1.1341913533372772, 1.106425002791019, 1.0682656195307005, 1.0540072137583119, 1.0321068155519082, 1.0088870160665773] 32
(32, 148)
(32, 148)
(32, 148)
loading_factors_matrix are similar


In [301]:
def varimax(Phi, gamma = 1.0, q = 100, tol = 1e-12):
    from numpy import eye, asarray, dot, sum, diag
    from numpy.linalg import svd
    p,k = Phi.shape
    R = eye(k)
    d=0
    for i in xrange(q):
        d_old = d
        Lambda = dot(Phi, R)
        u,s,vh = svd(dot(Phi.T,asarray(Lambda)**3 - (gamma/p) * dot(Lambda, diag(diag(dot(Lambda.T,Lambda))))))
        R = dot(u,vh)
        d = sum(s)
        #print i
        if d_old!=0 and d/d_old < 1 + tol: 
            print "convergence in: ", i, "iterations"
            break
    return dot(Phi, R)

In [419]:
loading_factors_matrix_rotated = varimax(loading_factors_matrix.T)
loading_factors_matrix_rotated_2 = varimax(loading_factors_matrix_2.T)
#loading_factors_matrix_rotated_df = pd.DataFrame(loading_factors_matrix_rotated.T)
#loading_factors_matrix_rotated_df
print loading_factors_matrix_rotated.shape
print loading_factors_matrix_rotated_2.shape

print sum(abs(loading_factors_matrix_rotated[:,0]))
print sum(sum(abs(loading_factors_matrix_rotated)))

print abs(loading_factors_matrix_rotated_2[:,0]).sum()
print abs(loading_factors_matrix_rotated_2).sum()
# convergence in:  86 iterations
# 380.83097886

convergence in:  65 iterations
convergence in:  65 iterations
(148, 32)
(148, 32)
27.1063163763
380.83076567
27.1063163763
380.83076567


In [28]:
# MLE method of factor Analysis
#!/usr/bin/env python
from __future__ import division

import numpy as np
from numpy import linalg as la

def factorAnalysis(x, numFactors, numIterations):    
    numData, numDim = x.shape
    mu = x.mean(axis=0)
    x -= mu
    
    cov =(x.T*x)/numData

    I = np.eye(numFactors)
    Psi = np.diag(np.diag(cov))
    scaling = np.power(la.det(cov), 1./numData)
    W = np.random.normal(0,np.sqrt(scaling/numFactors),(numDim,numFactors))

    oldL = -np.inf
    for i in xrange(numIterations):
        
        # Expectation step
        WPsi = np.dot(W.T,la.pinv(Psi)) 
        G = la.pinv(I+np.dot(WPsi,W))
    
        ez = np.dot(G,np.dot(WPsi,x.T)) 
        ezz = numData*G+np.dot(ez,ez.T)
    
        # Maximization step
        xEz = np.dot(x.T,ez.T)
        W = xEz*la.pinv(ezz)
        Psi = np.diag(np.diag(cov-np.dot(W,xEz.T)/numData))
        
        # Log Likelihood
        Sigma = W*W.T + Psi
        logSigma = np.log(la.det(Sigma))
        L = -numData/2*(np.trace(la.pinv(Sigma)*cov.T)+numDim*np.log(2*np.pi)+logSigma)
        
        if (L-oldL)<(1e-6):
            break
        oldL = L
        
    return W, Psi, mu

In [18]:
# Phi = W
def varimax(Phi, gamma = 1.0, q = 100, tol = 1e-6):
    from numpy import eye, asarray, dot, sum, diag
    from numpy.linalg import svd
    p,k = Phi.shape
    R = eye(k)
    d=0
    for i in xrange(q):
        d_old = d
        Lambda = dot(Phi, R)
        u,s,vh = svd(dot(Phi.T,asarray(Lambda)**3 - (gamma/p) * dot(Lambda, diag(diag(dot(Lambda.T,Lambda))))))
        R = dot(u,vh)
        d = sum(s)
        print i
        if d_old!=0 and d/d_old < 1 + tol: break
    return dot(Phi, R)

In [31]:
scores = np.asmatrix(np.genfromtxt("http://www-eio.upc.es/~jan/Data/BookOpenClosed.dat",delimiter="\t",skip_header=1))

In [53]:
print scores.shape
print X.shape
#import sympy as sp
#sp.init_printing()
#sp.Matrix(scores[0:10,:])

(88, 5)
(1980, 148)


In [56]:
[W, Psi, mu] = factorAnalysis(X, 32, 1000)

In [58]:
print W.shape
print W

(148, 32)
[[ 0.02119839  0.0716727   0.06022885 ...,  0.41741411  0.08772163
   0.40338747]
 [-0.09608771  0.22740218  0.07298899 ...,  0.49053622 -0.02268268
   0.46165969]
 [ 0.02011928 -0.05268804  0.06911309 ..., -0.30084462  0.08099182
  -0.39589764]
 ..., 
 [-0.26992739  0.1004103   0.0850555  ..., -0.03956153 -0.02884643
  -0.16202375]
 [ 0.01845861  0.04885697  0.34594024 ...,  0.01555451 -0.1268005
  -0.13848667]
 [-0.09412024  0.06978918  0.28337721 ..., -0.03319552 -0.16611678
  -0.17063972]]


In [29]:
# http://stackoverflow.com/questions/24560298/python-numpy-valueerror-operands-could-not-be-broadcast-together-with-shapes
[W, Psi, mu] = factorAnalysis(X, 32, 1000)

ValueError: operands could not be broadcast together with shapes (148,1980) (1980,148) 

In [51]:
X.T * X

ValueError: operands could not be broadcast together with shapes (148,1980) (1980,148) 

In [481]:
import os
import numpy as np
from numpy import eye, asarray, dot, sum, diag
from numpy.linalg import svd

# input raw data, center, apply weights
# (raw_data(i) - raw_data(mean)) * sqrt(weight)
# http://stats.stackexchange.com/questions/113485/weighted-principal-components-analysis

unweighted_filename = '/Users/pniessen/Rosetta_Desktop/Segmentation_2-point-0/sample_case_work/GoPro/gopro_raw_data_v1.csv'
weights_filename = '/Users/pniessen/Rosetta_Desktop/Segmentation_2-point-0/segmentr/web/gopro_weights.csv'

X_names = list(np.genfromtxt(unweighted_filename, delimiter=',', names=True).dtype.names)
X_unweighted = np.asmatrix(np.genfromtxt(unweighted_filename, delimiter=',', skip_header=1))
X_weights = np.asmatrix(np.genfromtxt(weights_filename, delimiter=',', skip_header=1))

X_unweighted_mean = (np.mean(X_unweighted, axis = 0))
X_unweighted_centered = X_unweighted - X_unweighted_mean
X_weighted = np.multiply(X_unweighted_centered.T,np.sqrt(X_weights))
X = X_weighted.T

#following http://sebastianraschka.com/Articles/2014_pca_step_by_step.html
# correlation matrix
X_cor_mat = np.corrcoef(X.T)

# get eigenvectors and eigenvalues for the from the correlation matrix
eig_val_cor, eig_vec_cor = np.linalg.eig(X_cor_mat)

# initialize empty loading_factor_matrix of (0 x length of eigenvector)
eig_vec_cors_length = eig_vec_cor[:,1].shape[0]
loading_factors_matrix_3 = np.asmatrix(np.zeros((0,eig_vec_cors_length))) # matrix shape must be tupple

# filter for eigvec > 1.0, generate loading factor, add to loading_factors_matrix
for i in range(len(eig_val_cor)):
    one_eigenvector = eig_vec_cor[:,i].reshape(1,eig_vec_cors_length).T # unit vector, direction
    one_eigenvalue = eig_val_cor[i] # scale
    
    if one_eigenvalue >=1:
        # http://stats.stackexchange.com/questions/143905/loadings-vs-eigenvectors-in-pca-when-to-use-one-or-another
        loading_factor = one_eigenvector * np.sqrt(one_eigenvalue)
        loading_factors_matrix_3 = np.r_[loading_factors_matrix_3,loading_factor.T]

print loading_factors_matrix_3.shape
print loading_factors_matrix_3.T[:,1].sum()
print abs(loading_factors_matrix_3.T).sum()
        
# now rotate loading_factor_matrix - this helps to filter out 'midrange' factors
# http://stackoverflow.com/questions/17628589/perform-varimax-rotation-in-python-using-numpy
def varimax(Phi, gamma = 1.0, q = 100, tol = 1e-10):
    
    print Phi, gamma, q, tol
    # gamma = 1.0: varimax
    # gamma = 0.0: quartimax  (https://github.com/rossfadely/consomme/blob/master/consomme/rotate_factor.py)

    p,k = Phi.shape
    R = eye(k) # identity matrix
    d=0
    for i in xrange(q):
        d_old = d
        Lambda = dot(Phi, R)
        u,s,vh = svd(dot(Phi.T,asarray(Lambda)**3 - (gamma/p) * dot(Lambda, diag(diag(dot(Lambda.T,Lambda))))))
        R = dot(u,vh)
        d = sum(s)
        #print i
        if d_old!=0 and d/d_old < 1 + tol: 
            print "convergence in: ", i, "iterations"
            break
    return dot(Phi, R)

loading_factors_matrix_rotated_3 = varimax(loading_factors_matrix_3.T)
print loading_factors_matrix_rotated_3.shape
print abs(loading_factors_matrix_rotated_3[:,0]).sum()
print abs(loading_factors_matrix_rotated_3).sum()

# tested vs SPSS output using gopro data
# for explanation of SPSS output
# see http://www.ats.ucla.edu/stat/spss/output/principal_components.htm
# ----------------
# SPSS loading factor matrix files:
# spss_unrotated: (148, 32) 468.803663037
# spss_rotated: (148, 32) sum: 381.386018769
# ----------------
# Homebrew loading factor matrix files:
# homebrew_unrotated: (148, 32) sum: 468.845148151
# homebrew_rotated: (148, 32) sum: 380.83076567
# -----pairwise comparison-------------
# unrotated loading_factors_matrix are similar using np.isclose @ atol = 1e-04
# rotated loading_factors_matrix are similar using np.isclose @ atol = 1e-04


(32, 148)
5.93393841669
468.845148151
[[ 0.10044908 -0.2573697  -0.11807879 ...,  0.08813666 -0.00421899
  -0.06133222]
 [ 0.10234864 -0.22576888 -0.23456041 ...,  0.0228426   0.06087528
  -0.12009664]
 [-0.28406691  0.37943922 -0.02530379 ..., -0.03923967 -0.00114615
   0.14932747]
 ..., 
 [-0.20858663  0.27096351  0.00158318 ..., -0.06873511 -0.01368568
  -0.29288004]
 [-0.56348231  0.12280602  0.30693165 ..., -0.06220677 -0.03616688
   0.03393988]
 [-0.40616038  0.30407462  0.16590372 ..., -0.10976092 -0.01455879
   0.02870455]] 1.0 100 1e-10
convergence in:  53 iterations
(148, 32)
27.1064098798
380.830519715


In [436]:
# now test vs SPSS output
spss_unrotated_filename = '/Users/pniessen/Rosetta_Desktop/Segmentation_2-point-0/segmentr/web/gopro_spss_unrotated_factor_matrix.csv'
loading_factors_spss_unrotated = np.asmatrix(np.genfromtxt(spss_unrotated_filename, delimiter=',', skip_header=1))

spss_rotated_filename = '/Users/pniessen/Rosetta_Desktop/Segmentation_2-point-0/segmentr/web/gopro_spss_rotated_factor_matrix.csv'
loading_factors_spss_rotated = np.asmatrix(np.genfromtxt(spss_rotated_filename, delimiter=',', skip_header=1))

loading_factors_spss_unrotated = loading_factors_spss_unrotated[:,1:] # remove row labels
print loading_factors_spss_unrotated.shape
loading_factors_spss_rotated = loading_factors_spss_rotated[:,1:] # remove row labels
print loading_factors_spss_rotated.shape



(148, 32)
(148, 32)


In [465]:
# test to see if methods are same
print 'SPSS loading factor matrix files:'
print 'spss_unrotated:', loading_factors_spss_unrotated.shape, abs(loading_factors_spss_unrotated).sum()
print 'spss_rotated:', loading_factors_spss_rotated.shape, 'sum:', abs(loading_factors_spss_rotated).sum()
print '----------------'
print 'Homebrew loading factor matrix files:'
print 'homebrew_unrotated:', loading_factors_matrix_3.T.shape, 'sum:', abs(loading_factors_matrix_3).sum()
print 'homebrew_rotated:', loading_factors_matrix_rotated_3.shape, 'sum:', abs(loading_factors_matrix_rotated_3).sum()
print '-----pairwise comparison-------------'

# http://stackoverflow.com/questions/25962838/check-if-float-is-close-to-any-float-stored-in-array
if np.isclose(loading_factors_spss_unrotated, loading_factors_matrix_3.T).any():
    print "unrotated loading_factors_matrix are similar using np.isclose"
    
if np.isclose(loading_factors_spss_rotated, loading_factors_matrix_rotated_3, atol=1e-04).any():
    print "rotated loading_factors_matrix are similar using np.isclose"

SPSS loading factor matrix files:
spss_unrotated: (148, 32) 468.803663037
spss_rotated: (148, 32) sum: 381.386018769
----------------
Homebrew loading factor matrix files:
homebrew_unrotated: (148, 32) sum: 468.845148151
homebrew_rotated: (148, 32) sum: 380.83076567
-----pairwise comparison-------------
unrotated loading_factors_matrix are similar using np.isclose
rotated loading_factors_matrix are similar using np.isclose


In [473]:
type(loading_factors_matrix_rotated_3)

numpy.matrixlib.defmatrix.matrix

In [475]:
z = np.asarray(loading_factors_matrix_rotated_3)

In [477]:
z.shape
type(z)

numpy.ndarray